# 第五次作业_02_6.4

> 新雅62/CDIE6
> 2016013327 项雨桐

## 0. 导入包和数据

In [ ]:
from skimage.io import ImageCollection
import numpy as np
import numpy.random as rand
import random
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.metrics import accuracy_score,confusion_matrix,roc_auc_score,roc_curve
import matplotlib.pyplot as plt

In [ ]:
rand.seed(15)
path = 'HomeworkData/Pictures/'

## 1. 数据准备

In [ ]:
def image_read(index):
    full_path = path + str(index) + '/*.png' 
    data = ImageCollection(full_path, as_gray=True).concatenate()
    data = data.reshape(-1,(data.shape[1]*data.shape[2]))
    data = np.concatenate((data, index * np.ones((data.shape[0],1))), axis = 1)
    return data

def data_preparation(data_all, num):
    data_sam = random.sample(data_all, num)
    data_sam = np.asarray(data_sam)
    data_sam = data_sam.reshape((data_sam.shape[0]*data_sam.shape[1]),-1)
    data_X = data_sam[:,:-1]
    data_y = data_sam[:,-1]
    X_train, X_test, y_train, y_test = train_test_split(data_X, data_y, train_size=0.75)
    return X_train, X_test, y_train, y_test

In [ ]:
data_0 = image_read(0)
data_1 = image_read(1)
data_2 = image_read(2)
data_3 = image_read(3)
data_4 = image_read(4)
data_5 = image_read(5)
data_6 = image_read(6)
data_7 = image_read(7)
data_8 = image_read(8)
data_9 = image_read(9)

data_all = [data_0, data_1 ,data_2 ,data_3 ,data_4 ,data_5 ,data_6 ,data_7 ,data_8 ,data_9]

X_2_train, X_2_test, y_2_train, y_2_test = data_preparation(data_all,2)
X_5_train, X_5_test, y_5_train, y_5_test = data_preparation(data_all,5)
X_10_train, X_10_test, y_10_train, y_10_test = data_preparation(data_all,10)

## 2. 模型预测

In [ ]:
model_2 = KNN().fit(X_2_train,y_2_train)
model_5 = KNN().fit(X_5_train,y_5_train)
model_10 = KNN().fit(X_10_train,y_10_train)

y_2_pred = model_2.predict(X_2_test)
y_2_prob = (model_2.predict_proba(X_2_test)[:,1]).ravel()
y_5_pred = model_5.predict(X_5_test)
y_10_pred = model_10.predict(X_10_test)

## 3. 模型评价

In [ ]:
acc_2 = accuracy_score(y_2_test,y_2_pred)
acc_5 = accuracy_score(y_5_test,y_5_pred)
acc_10 = accuracy_score(y_10_test,y_10_pred)

TN, FP, FN, TP = confusion_matrix(y_2_test,y_2_pred).ravel()
TPR = TP/(TP+FN)
FPR = FP/(FP+TN)
TNR = TN/(TN+FP)
FNR = FN/(FN+TP)
sensitivity = TP/(TP+FN)
specificity = TN/(TN+FP)
FDR = FP/(FP+TP)

print("测试准确率:")
print("2分类: " + str(round(acc_2, 4)))
print("5分类: " + str(round(acc_5, 4)))
print("10分类: " + str(round(acc_10, 4)))
print("2分类性能参数:")
print("TPR: " + str(round(TPR, 4)))
print("FPR: " + str(round(FPR, 4)))
print("TNR: " + str(round(TNR, 4)))
print("FNR: " + str(round(FNR, 4)))
print("sensitivity: " + str(round(sensitivity, 4)))
print("specificity: " + str(round(specificity, 4)))
print("FDR: " + str(round(FDR, 4)))

In [ ]:
AUC = roc_auc_score(y_2_test, y_2_prob)
print("AUC: " + str(round(AUC, 4)))

fpr,tpr,th = roc_curve(y_2_test, y_2_prob, pos_label = max(y_2_test))
plt.title('ROC')
plt.plot(fpr,tpr,'b')
plt.ylabel('TPR')
plt.xlabel('FPR')
plt.show()

## 4. 不同k值

In [ ]:
model_10_1 = KNN(n_neighbors=1).fit(X_10_train,y_10_train)
model_10_3 = KNN(n_neighbors=3).fit(X_10_train,y_10_train)
model_10_5 = KNN().fit(X_10_train,y_10_train)
model_10_7 = KNN(n_neighbors=7).fit(X_10_train,y_10_train)
model_10_9 = KNN(n_neighbors=9).fit(X_10_train,y_10_train)

In [ ]:
print("k=1: ", model_10_1.score(X_10_test,y_10_test))
print("k=3: ", model_10_3.score(X_10_test,y_10_test))
print("k=5: ", model_10_5.score(X_10_test,y_10_test))
print("k=7: ", model_10_7.score(X_10_test,y_10_test))
print("k=9: ", model_10_9.score(X_10_test,y_10_test))

## 5. 不同距离度量方式

In [ ]:
model_10_euc = KNN(n_neighbors=1).fit(X_10_train,y_10_train)
model_10_man = KNN(n_neighbors=1, p=1).fit(X_10_train,y_10_train)
model_10_che = KNN(n_neighbors=1, metric="chebyshev").fit(X_10_train,y_10_train)
model_10_cos = KNN(n_neighbors=1, metric="cosine").fit(X_10_train,y_10_train)

In [ ]:
print("euclidean: ", model_10_euc.score(X_10_test,y_10_test))
print("manhattan: ", model_10_man.score(X_10_test,y_10_test))
print("chebyshev: ", model_10_che.score(X_10_test,y_10_test))
print("cosine: ", model_10_cos.score(X_10_test,y_10_test))